In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)

# 맥도날드 메뉴 리스트 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 상세 페이지로 진입 (JavaScript 함수 실행)
driver.execute_script("goDetail(689)")  # Double McSpicy® Shanghai Burger
time.sleep(2)

try:
    toggle_button = driver.find_element(By.XPATH, "//div[@class='toggle']//button[@aria-controls='toggle02']")
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_button)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", toggle_button)
    time.sleep(1.5)
except Exception as e:
    print("❌ 영양정보 버튼 클릭 실패:", e)

# 테이블 추출
try:
    row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
    cells = row.find_elements(By.TAG_NAME, "td")
    data = {
        "중량(g)": cells[0].text.replace("g", "").strip(),
        "중량(ml)": cells[1].text.replace("ml", "").strip(),
        "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
        "당(g)": cells[3].text.replace("g", "").strip(),
        "단백질(g)": cells[4].text.replace("g", "").strip(),
        "포화지방(g)": cells[5].text.replace("g", "").strip(),
        "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
        "카페인(mg)": cells[7].text.replace("mg", "").strip()
    }
    print("✅ 수집 완료:", data)
except Exception as e:
    print("❌ 영양정보 테이블 수집 실패:", e)

✅ 수집 완료: {'중량(g)': '341', '중량(ml)': '-', '열량(kcal)': '807', '당(g)': '15', '단백질(g)': '48', '포화지방(g)': '18', '나트륨(mg)': '1115', '카페인(mg)': '-'}


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)

# 페이지 접속 및 '버거' 진입
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 상세 진입
driver.execute_script("goDetail(689)")  # 첫 버거
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명 가져오기
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 영양정보 버튼 클릭
        toggle_button = driver.find_element(By.XPATH, "//div[@class='toggle']//button[@aria-controls='toggle02']")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", toggle_button)
        time.sleep(1.5)

        # 테이블 수집
        row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
        cells = row.find_elements(By.TAG_NAME, "td")
        data = {
            
            "중량(g)": cells[0].text.replace("g", "").strip(),
            "중량(ml)": cells[1].text.replace("ml", "").strip(),
            "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
            "당(g)": cells[3].text.replace("g", "").strip(),
            "단백질(g)": cells[4].text.replace("g", "").strip(),
            "포화지방(g)": cells[5].text.replace("g", "").strip(),
            "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
            "카페인(mg)": cells[7].text.replace("mg", "").strip()
        }
        print(f"✅ {menu_name} 수집 완료")
        data_list.append(data)

    except Exception as e:
        print("❌ 영양정보 수집 실패:", e)

    # 다음 버튼 클릭 시도
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
        time.sleep(3)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(3)
    except Exception as e:
        print("🚨 다음 버튼 없음 or 마지막 메뉴입니다. 종료합니다.")
        break

# 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
print("🍔 모든 수집 완료!")
driver.quit()


❌ 영양정보 수집 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [0x00007FF766FAD2D4+11

In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 맥도날드 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 진입
driver.execute_script("goDetail(689)")
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 1. 영양정보 버튼 스크롤 + 클릭
        toggle_btn = driver.find_element(By.XPATH, "//div[@class='toggle']//button[@aria-controls='toggle02']")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_btn)
        time.sleep(0.8)
        driver.execute_script("arguments[0].click();", toggle_btn)

        # 2. 열릴 때까지 대기
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")))
        time.sleep(1)  # 안정화를 위한 짧은 딜레이

        # 3. 테이블 수집
        row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
        cells = row.find_elements(By.TAG_NAME, "td")

        data = {
            "메뉴": menu_name,
            "중량(g)": cells[0].text.replace("g", "").strip(),
            "중량(ml)": cells[1].text.replace("ml", "").strip(),
            "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
            "당(g)": cells[3].text.replace("g", "").strip(),
            "단백질(g)": cells[4].text.replace("g", "").strip(),
            "포화지방(g)": cells[5].text.replace("g", "").strip(),
            "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
            "카페인(mg)": cells[7].text.replace("mg", "").strip()
        }
        print(f"✅ {menu_name} 수집 완료")
        data_list.append(data)

    except Exception as e:
        print("❌ 영양정보 수집 실패:", e)

    # 다음 버튼 클릭
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)

    except Exception:
        print("🚨 더 이상 다음 메뉴 없음! 종료")
        break

# 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
driver.quit()
print("🍔 모든 수집 완료 및 저장 완료!")


❌ 영양정보 수집 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [0x00007FF766FAD2D4+11

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 맥도날드 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 진입
driver.execute_script("goDetail(689)")
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 1. 영양정보 버튼 스크롤 + 클릭
        toggle_btn = driver.find_element(By.XPATH, "//div[@class='toggle']//button[@aria-controls='toggle02']")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_btn)
        time.sleep(0.8)
        driver.execute_script("arguments[0].click();", toggle_btn)

        # 2. 열릴 때까지 대기
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")))
        time.sleep(1)  # 안정화를 위한 짧은 딜레이

        # 3. 테이블 수집
        row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
        cells = row.find_elements(By.TAG_NAME, "td")

        data = {
            "메뉴": menu_name,
            "중량(g)": cells[0].text.replace("g", "").strip(),
            "중량(ml)": cells[1].text.replace("ml", "").strip(),
            "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
            "당(g)": cells[3].text.replace("g", "").strip(),
            "단백질(g)": cells[4].text.replace("g", "").strip(),
            "포화지방(g)": cells[5].text.replace("g", "").strip(),
            "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
            "카페인(mg)": cells[7].text.replace("mg", "").strip()
        }
        print(f"✅ {menu_name} 수집 완료")
        data_list.append(data)

    except Exception as e:
        print("❌ 영양정보 수집 실패:", e)

    # 다음 버튼 클릭
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)

    except Exception:
        print("🚨 더 이상 다음 메뉴 없음! 종료")
        break

# 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
driver.quit()
print("🍔 모든 수집 완료 및 저장 완료!")


❌ 영양정보 수집 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [0x00007FF766FAD2D4+11

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 맥도날드 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 진입
driver.execute_script("goDetail(689)")
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 영양정보 버튼 클릭 시도
        toggle_btn = driver.find_element(By.XPATH, "//div[@class='toggle']//button[@aria-controls='toggle02']")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_btn)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", toggle_btn)

        # 테이블이 뜰 때까지 기다림
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")))
            time.sleep(1)

            # 수집
            row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
            cells = row.find_elements(By.TAG_NAME, "td")

            data = {
                "메뉴": menu_name,
                "중량(g)": cells[0].text.replace("g", "").strip(),
                "중량(ml)": cells[1].text.replace("ml", "").strip(),
                "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
                "당(g)": cells[3].text.replace("g", "").strip(),
                "단백질(g)": cells[4].text.replace("g", "").strip(),
                "포화지방(g)": cells[5].text.replace("g", "").strip(),
                "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
                "카페인(mg)": cells[7].text.replace("mg", "").strip()
            }

            data_list.append(data)
            print(f"✅ {menu_name} 수집 완료")

        except TimeoutException:
            print(f"⚠️ [수집 실패] '{menu_name}' - 영양성분 테이블이 표시되지 않음")

    except Exception as e:
        print(f"❌ [예상치 못한 에러 - 메뉴 진입 또는 버튼 클릭 실패]: {e}")

    # ✅ 다음 버튼은 무조건 눌러야 함
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
    except Exception:
        print("🚨 다음 버튼 없음. 모든 수집 완료!")
        break

# CSV 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
driver.quit()
print("🍔 모든 수집 완료 및 저장 완료!")


❌ [예상치 못한 에러 - 메뉴 진입 또는 버튼 클릭 실패]: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 진입
driver.execute_script("goDetail(689)")
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명 수집
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 1. 영양정보 버튼 스크롤
        toggle_btn_xpath = "//div[@class='toggle']//button[@aria-controls='toggle02']"
        toggle_btn = driver.find_element(By.XPATH, toggle_btn_xpath)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_btn)

        # 2. 버튼 클릭 가능한 상태까지 기다린 후 클릭
        wait.until(EC.element_to_be_clickable((By.XPATH, toggle_btn_xpath))).click()
        time.sleep(1.2)

        # 3. 테이블 뜰 때까지 기다림
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")))

        # 4. 영양정보 추출
        row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
        cells = row.find_elements(By.TAG_NAME, "td")

        data = {
            "메뉴": menu_name,
            "중량(g)": cells[0].text.replace("g", "").strip(),
            "중량(ml)": cells[1].text.replace("ml", "").strip(),
            "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
            "당(g)": cells[3].text.replace("g", "").strip(),
            "단백질(g)": cells[4].text.replace("g", "").strip(),
            "포화지방(g)": cells[5].text.replace("g", "").strip(),
            "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
            "카페인(mg)": cells[7].text.replace("mg", "").strip()
        }

        data_list.append(data)
        print(f"✅ {menu_name} 수집 완료")

    except TimeoutException:
        print(f"⚠️ [수집 실패] '{menu_name}' - 영양성분 테이블이 표시되지 않음")

    except Exception as e:
        print(f"❌ [예상치 못한 에러]: {e}")

    # 다음 버튼 클릭 시도
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
    except Exception:
        print("🚨 다음 버튼 없음. 모든 수집 완료!")
        break

# CSV 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
driver.quit()
print("🍔 모든 수집 완료 및 저장 완료!")


❌ [예상치 못한 에러]: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [0x00007FF766FAD2D4+1

In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

# 드라이버 설정
service = Service(executable_path=r"C:\Users\user\Documents\GitHub\main\code\visualization\chromedriver-win64\chromedriver.exe")
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 페이지 접속
driver.get("https://www.mcdonalds.co.kr/kor/menu/list.do")
time.sleep(3)

# '버거' 탭 클릭
driver.find_element(By.LINK_TEXT, "버거").click()
time.sleep(2)

# 첫 번째 버거 진입
driver.execute_script("goDetail(689)")
time.sleep(2)

data_list = []

while True:
    try:
        # 메뉴명 수집
        menu_name = driver.find_element(By.CSS_SELECTOR, "div.innerView h2.tit").text.strip()

        # 1. 영양정보 버튼 스크롤
        toggle_btn_xpath = "//div[@class='toggle']//button[@aria-controls='toggle02']"
        toggle_btn = driver.find_element(By.XPATH, toggle_btn_xpath)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", toggle_btn)

        # 2. 버튼 클릭 가능한 상태까지 기다린 후 클릭
        wait.until(EC.element_to_be_clickable((By.XPATH, toggle_btn_xpath))).click()
        time.sleep(1.2)

        # 3. 테이블 뜰 때까지 기다림
        wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")))

        # 4. 영양정보 추출
        row = driver.find_element(By.XPATH, "//table[contains(@class, 'nutrDesc')]/tbody/tr[1]")
        cells = row.find_elements(By.TAG_NAME, "td")

        data = {
            "메뉴": menu_name,
            "중량(g)": cells[0].text.replace("g", "").strip(),
            "중량(ml)": cells[1].text.replace("ml", "").strip(),
            "열량(kcal)": cells[2].text.replace("kcal", "").strip(),
            "당(g)": cells[3].text.replace("g", "").strip(),
            "단백질(g)": cells[4].text.replace("g", "").strip(),
            "포화지방(g)": cells[5].text.replace("g", "").strip(),
            "나트륨(mg)": cells[6].text.replace("mg", "").strip(),
            "카페인(mg)": cells[7].text.replace("mg", "").strip()
        }

        data_list.append(data)
        print(f"✅ {menu_name} 수집 완료")

    except TimeoutException:
        print(f"⚠️ [수집 실패] '{menu_name}' - 영양성분 테이블이 표시되지 않음")

    except Exception as e:
        print(f"❌ [예상치 못한 에러]: {e}")

    # 다음 버튼 클릭 시도
    # try:
    #     next_btn = driver.find_element(By.CSS_SELECTOR, "a.arrow.next")
    #     driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant'});", next_btn)
    #     time.sleep(1)
    #     driver.execute_script("arguments[0].click();", next_btn)
    #     time.sleep(2)
    # except Exception:
    #     print("🚨 다음 버튼 없음. 모든 수집 완료!")
        break

# CSV 저장
df = pd.DataFrame(data_list)
df.to_csv("맥도날드_버거_영양정보.csv", index=False, encoding="utf-8-sig")
driver.quit()
print("🍔 모든 수집 완료 및 저장 완료!")


❌ [예상치 못한 에러]: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.innerView h2.tit"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF766FA5355+78597]
	GetHandleVerifier [0x00007FF766FA53B0+78688]
	(No symbol) [0x00007FF766D591AA]
	(No symbol) [0x00007FF766DAF149]
	(No symbol) [0x00007FF766DAF3FC]
	(No symbol) [0x00007FF766E02467]
	(No symbol) [0x00007FF766DD712F]
	(No symbol) [0x00007FF766DFF2BB]
	(No symbol) [0x00007FF766DD6EC3]
	(No symbol) [0x00007FF766DA03F8]
	(No symbol) [0x00007FF766DA1163]
	GetHandleVerifier [0x00007FF76724EF0D+2870973]
	GetHandleVerifier [0x00007FF7672496B8+2848360]
	GetHandleVerifier [0x00007FF767266993+2967875]
	GetHandleVerifier [0x00007FF766FC019A+188746]
	GetHandleVerifier [0x00007FF766FC847F+222255]
	GetHandleVerifier [0x00007FF766FAD2D4+1